# Exploring propagation of record changes across databases and annotations

In [2]:
# For each record in SwissProt
#    Extract Taxa
#    Extract ProteinName and Subname
#    Extract 

In [4]:
# Imports
from Bio import SeqIO
import gzip

In [12]:
# Index UniProt proteins so that we can access via their accessions. 
#  - Only needs to be run once. Takes a few minutes. 

data_dir = '../data/'
up_gz = data_dir+'uniprot_sprot.dat.bgz'
up_db = data_dir+'uniprot_sprot.dat.biopy_db'
uniprot = SeqIO.index_db(index_filename=up_db, filenames=up_gz, format="swiss")


In [13]:
# Index gb_files
gb_file=data_dir+"gbbct60.seq.gz"
#h = gzip.open(gb_file, "rt")
#r =SeqIO.parse(h, "gb").__next__()

In [26]:
# Create a mapping from UniProt to GenBank and RefSeq. 
#gb_sp_map={id.split(":")[1]:r  for r in uniprot for id in uniprot[r].dbxrefs if id[0:4]=="EMBL"}
#rs_sp_map={id.split(":")[1]:r  for r in uniprot for id in uniprot[r].dbxrefs if id[0:6]=="RefSeq"}
def check_db(r):
    return any([id[0:6]=="RefSeq" or id[0:4]=="EMBL" for id in r.dbxrefs])

def db_tuple(r):
    embl=(None,None)
    rs=(None,None)
    for db_acc in r.dbxrefs:
        (db,acc)=db_acc.split(":")
        if db=="RefSeq":
            rs=(acc,r)
        if db=="EMBL":
            embl=(acc,r)
    return(embl,rs)


dbxref_tuples=[db_tuple(uniprot[acc])  for acc in uniprot if check_db(uniprot[acc]) ]
gb_sp_map={acc:r for ((acc,r),_) in dbxref_tuples if acc is not None}
gb_sp_map={acc:r for (_,(acc,r)) in dbxref_tuples if acc is not None}


In [48]:

gbs_iter=SeqIO.parse(h, "gb")
gbs_cut=[gbs_iter.__next__() for i in range(100)]
prots=([ (p.qualifiers['product'],p.qualifiers['protein_id'], p.qualifiers['translation']) for r in gbs_cut for p in r.features if p.type=="CDS" and 'pseudo' not in p.qualifiers])
I=[p[1][0] in gb_sp_map or p[1][0].split('.')[0] in gb_sp_map for p in prots]
any(i)

KeyboardInterrupt: 

In [ ]:
# For each set of genbank records (gb*.seq.gz)
#  Extract record
#  Download history
#  